In [1]:
library(data.table)
library(magrittr)
library(ggplot2)

mydata <- fread("https://raw.githubusercontent.com/arunsrinivasan/satrdays-workshop/master/flights_2014.csv")

## Performance Comparison
You can compare performance of the filtering process <b>(With or Without KEY).</b><br />


In [2]:
system.time(mydata[origin %in% c("JFK", "LGA")])


   user  system elapsed 
  0.072   0.004   0.076 

In [3]:
setkey(mydata,origin)
system.time(mydata[c("JFK", "LGA")])

   user  system elapsed 
  0.020   0.000   0.018 


<table align="center" cellpadding="0" cellspacing="0" class="tr-caption-container" style="margin-left: auto; margin-right: auto; text-align: center;"><tbody>
<tr><td style="text-align: center;"><a href="https://2.bp.blogspot.com/-cNI65X2y9_c/WAeNAnOf54I/AAAAAAAAFiU/mgen0LJVzXkn0_DHd5ibxExSudLhAwWtwCLcB/s1600/Performance%2Bdata.table.png" imageanchor="1" style="margin-left: auto; margin-right: auto;"><img border="0" src="https://2.bp.blogspot.com/-cNI65X2y9_c/WAeNAnOf54I/AAAAAAAAFiU/mgen0LJVzXkn0_DHd5ibxExSudLhAwWtwCLcB/s1600/Performance%2Bdata.table.png" /></a></td></tr>
<tr><td class="tr-caption" style="text-align: center;">Performance - With or without KEY</td></tr>
</tbody></table>


If you look at the real time in the image above, setting key makes filtering twice as faster than without using keys.
<br />
We can also set keys to multiple columns like we did below to columns 'origin' and 'dest'. See the example below.

In [4]:
setkey(mydata, origin, dest)

### Filtering while setting keys on Multiple Columns

In [5]:

# First key column 'origin' matches 'JFK and second key column 'dest' matches &#8220;MIA&#8221;<br />
mydata[.("JFK", "MIA")]

It is equivalent to the following code :

In [6]:
mydata[origin == "JFK" & dest == "MIA"]

### To identify the column(s) indexed by

In [7]:
key(mydata)

[1] "origin" "dest"


<b>Result :</b> It returns origin and dest as these are columns that are set keys.</div>


## Sorting Data

We can sort data using <b>``setorder()``</b> function, By default, it sorts data on ascending order.<br />


In [8]:
mydata01 = setorder(mydata, origin)

### Sorting Data on descending order 
In this case, we are sorting data by 'origin' variable on descending order.

In [9]:
mydata02 = setorder(mydata, -origin)

### Sorting Data based on multiple variables
In this example, we tells R to reorder data first by origin on ascending order and then variable 'carrier'on descending order.

In [10]:
mydata03 = setorder(mydata, origin, -carrier)

### In-place Assignment

Easy to efficiently add and remove columns


In [11]:
test.dt <- data.table(x = 1:3, y = (1:3)^2)
test.dt

In [12]:
test.dt[, z := x * 2]
test.dt %>% print() 

   x y z
1: 1 1 2
2: 2 4 4
3: 3 9 6


In [13]:
test.dt[, z := NULL]
test.dt %>% print()

   x y
1: 1 1
2: 2 4
3: 3 9


## Adding Columns (Calculation on rows)
You can do any operation on rows by adding <b>``:=`` operator</b>. 
In this example, we are subtracting 'dep_delay' variable 
from 'dep_time' variable to compute scheduled departure time.


In [14]:
mydata[, dep_sch:=dep_time - dep_delay]

## Adding Multiple Columns

In [15]:
mydata002 = mydata[, c("dep_sch","arr_sch"):=list(dep_time - dep_delay, arr_time - arr_delay)]

## IF THEN ELSE

The 'IF THEN ELSE' conditions are very popular for recoding values. In data.table package, it can be done with the following methods :

* <b>Method I : &nbsp; &nbsp;</b>mydata[, flag:= 1*(min &lt; 50)]<br />
* <b>Method II :</b>&nbsp; &nbsp;mydata[, flag:= ifelse(min &lt; 50, 1,0)]<br />

It means to set flag= 1 if min is less than 50. Otherwise, set flag =0.


## How to write Sub Queries (like SQL)

We can use this format - <b>DT[ ] [ ] [ ] </b>to build a chain in data.table. 
It is like sub-queries like SQL.


In [16]:
mydata[, dep_sch:=dep_time - dep_delay][,.(dep_time,dep_delay,dep_sch)]


First, we are computing scheduled departure time and then selecting only relevant columns.

### Summarize or Aggregate Columns

Like SAS PROC MEANS procedure, we can generate summary statistics of specific variables. In this case, we are calculating mean, median, minimum and maximum value of variable arr_delay.<br />
<blockquote class="tr_bq">
mydata[, .(mean = mean(arr_delay, na.rm = TRUE),<br />
median = median(arr_delay, na.rm = TRUE),<br />
min = min(arr_delay, na.rm = TRUE),<br />
max = max(arr_delay, na.rm = TRUE))]</blockquote>


<table align="center" cellpadding="0" cellspacing="0" class="tr-caption-container" style="margin-left: auto; margin-right: auto;"><tbody>
<tr><td style="text-align: center;"><a href="https://1.bp.blogspot.com/-TvbAiwlS0QY/WAeYoCr7lgI/AAAAAAAAFik/aR6CXMEpi8oXAWp3amjHepdNwNjsiQO_gCLcB/s1600/Summary.png" imageanchor="1" style="margin-left: auto; margin-right: auto; text-align: center;"><img border="0" src="https://1.bp.blogspot.com/-TvbAiwlS0QY/WAeYoCr7lgI/AAAAAAAAFik/aR6CXMEpi8oXAWp3amjHepdNwNjsiQO_gCLcB/s1600/Summary.png" /></a></td></tr>
<tr><td class="tr-caption" style="text-align: center;">Summarize with data.table package</td></tr>
</tbody></table>


<b><span style="font-size: large;">Summarize Multiple Columns</span></b><br />
<br />
To summarize multiple variables, we can simply write all the summary statistics function in a bracket. See the command below-<br />

In [18]:

mydata[, .(mean(arr_delay), mean(dep_delay))]

If you need to calculate summary statistics for a larger list of variables, you can use <b>.SD and .SDcols</b> operators. 
The <b>.SD</b>&nbsp;operator implies <b>'Subset of Data'.&nbsp;</b><br />
In this case, we are calculating mean of two variables - arr_delay and dep_delay.<br />

In [19]:

mydata[, lapply(.SD, mean), .SDcols = c("arr_delay", "dep_delay")]


### Use key column in a by operation 
Instead of 'by', you can use <b>``keyby=``</b>operator.

In [20]:

mydata[, .(mean_arr_delay = mean(arr_delay, na.rm = TRUE)),
           keyby = origin]

### Summarize multiple variables by group 'origin'


In [21]:
mydata[, .(mean(arr_delay, na.rm = TRUE), 
           mean(dep_delay, na.rm = TRUE)), 
           by = origin]


Or it can be written like below: 



In [22]:
mydata[, lapply(.SD, mean, na.rm = TRUE), .SDcols = c("arr_delay", "dep_delay"), by = origin]

### Remove Duplicates

You can remove non-unique / duplicate cases with unique() function. 
Suppose you want to eliminate duplicates based on a variable, say. carrier.<br />


In [23]:

setkey(mydata, "carrier")
unique(mydata)

Suppose you want to remove duplicated based on all the variables. You can use the command below:


In [24]:

setkey(mydata, NULL)
unique(mydata)


<b>Note : </b>Setting key to NULL is not required if no key is already set.

### Extract values within a group

The following command selects first and second values from a categorical variable carrier.


In [25]:
mydata[, .SD[1:2], by=carrier]

### Select LAST value from a group


In [26]:
mydata[, .SD[.N], by=carrier]

### SQL's RANK OVER PARTITION

* In SQL, Window functions are very useful for solving complex data problems. 
* RANK OVER PARTITION is the most popular window function. 
* It can be easily translated in data.table with the help of <b>``frank()``</b> function. 
* <b>``frank()``</b> is similar to base R's <b>``rank()``</b> function but much faster. 
* See the code below.<br />


In [27]:
dt = mydata[, rank:= frank(-distance,ties.method = "min"), by=carrier]


In this case, we are calculating rank of variable 'distance' by 'carrier'. 
We are assigning rank 1 to the highest value of 'distance' within unique values of 'carrier'.

## Cumulative SUM by GROUP

We can calculate cumulative sum by using <b>cumsum()</b> function.<br />


In [28]:
dat = mydata[, cum:=cumsum(distance), by=carrier]

## Lag and Lead
* The lag and lead of a variable can be calculated with ``shift()`` function. 
* The syntax of ``shift()`` function is as follows: <pre><code>shift(variable_name, number_of_lags, type=c("lag", "lead"))
</code></pre>


In [29]:
DT <- data.table(A=1:5)

DT[ , X := shift(A, 1, type="lag")]
DT[ , Y := shift(A, 1, type="lead")]



<table align="center" cellpadding="0" cellspacing="0" class="tr-caption-container" style="margin-left: auto; margin-right: auto; text-align: center;"><tbody>
<tr><td style="text-align: center;"><a href="https://3.bp.blogspot.com/-Jz3wnUM0A-g/WAerXNB1x5I/AAAAAAAAFjA/vYRXZpGj3TkX4-tnhqbvIck6uG4iMsgKQCLcB/s1600/Lag%2Band%2Blead.png" imageanchor="1" style="margin-left: auto; margin-right: auto;"><img border="0" src="https://3.bp.blogspot.com/-Jz3wnUM0A-g/WAerXNB1x5I/AAAAAAAAFjA/vYRXZpGj3TkX4-tnhqbvIck6uG4iMsgKQCLcB/s1600/Lag%2Band%2Blead.png" /></a></td></tr>
<tr><td class="tr-caption" style="text-align: center;">Lag and Lead Function</td></tr>
</tbody></table>
<div>

## Between and LIKE Operator

We can use ``%between%`` operator to define a range. It is inclusive of the values of both the ends.


In [30]:

DT = data.table(x=6:10)
DT[x %between% c(7,9)]

The %like% is mainly used to find all the values that matches a pattern.


In [32]:
DT = data.table(Name=c("dep_time","dep_delay","arrival"), ID=c(2,3,4))
DT[Name %like% "dep"]